In [7]:
import sys
import os
sys.path.append(os.path.join(os.path.abspath(''), '../'))

from evaluation.evaluation_harness import OnlineEvaluator, include_coldstart, include_all, score_time_spent, score_constant
from statistical_test import estimate_probability_each_model_is_best
from pprint import pprint
import pandas as pd
sys.path.append(os.path.join(os.path.abspath(''), '../backend'))
from backend.blueprints.recommendation import model_wrappers

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
evaluator = OnlineEvaluator(include_all, score_constant)
all_metrics = []
def run_eval(model_name, model_save_path):
    evaluator.reset(model_name, model_save_path)
    # evaluator.plot_top_N_hit_percentage_percentiles(10)
    evaluator.compute_top_N_hit_percentage(10)
    # evaluator.plot_top_N_hit_percentage_percentiles(50)
    evaluator.compute_top_N_hit_percentage(50)
    # evaluator.plot_user_rank_roc_curve()
    evaluator.compute_user_rank_auc_roc()
    # evaluator.save_metrics('test_online_evaluator', overwrite=True)
    # pprint(evaluator.metrics)
    all_metrics.append({'model_name': model_name, 'model_save_path': model_save_path, **evaluator.metrics})
    # print()
evaluator.all_results.sort_values('timestamp', inplace=True)
evaluator.all_results.to_csv("test.csv", index=False)

In [14]:
current_models = []
for model_wrapper in model_wrappers:
    model_wrapper.model = model_wrapper.definition()
    model_wrapper.model.load(model_wrapper.model_save_file_name, load_published_model=True)
    current_models.append((model_wrapper.model.name(), model_wrapper.model_save_file_name))
print(current_models)
# groups = evaluator.all_results.groupby(by=["rec_model_name", "rec_model_save_path"])
# for (model_name, model_save_path), data in groups:
all_metrics = []
for model_name, model_save_path in current_models:
    run_eval(model_name, model_save_path)
results = pd.DataFrame(all_metrics)

[('common_neighbors', 'evaluation_test_common_neighbors_constant_scoring'), ('common_neighbors', 'evaluation_test_common_neighbors_percentile_scoring'), ('common_neighbors', 'evaluation_test_common_neighbors'), ('game_popularity', 'evaluation_test_popularity_model'), ('random', 'evaluation_test_random_model'), ('neural_collborative_filtering_cf', 'evaluation_test_cf_low_weight_decay_increased_lr'), ('neural_collborative_filtering_gcf', 'evaluation_test_gcf_low_weight_decay_increased_lr'), ('neural_collborative_filtering_mlp', 'evaluation_test_mlp_low_weight_decay_increased_lr'), ('neural_collborative_filtering_ncf', 'evaluation_test_ncf_low_weight_decay_increased_lr'), ('neural_collborative_filtering_cf', 'evaluation_test_cf_embed_all_except_tags_genres'), ('neural_collborative_filtering_gcf', 'evaluation_test_gcf_embed_all_except_tags_genres'), ('neural_collborative_filtering_mlp', 'evaluation_test_mlp_embed_all_except_tags_genres'), ('neural_collborative_filtering_ncf', 'evaluation_t

In [15]:
estimate_probability_each_model_is_best(results, 'user_rank_auc_roc')

In [16]:
evaluation_names = {'evaluation_test_cf_low_weight_decay_increased_lr': 'collborative_filter', 'evaluation_test_gcf_low_weight_decay_increased_lr': 'non_linear_collaborative_filter', 'evaluation_test_mlp_low_weight_decay_increased_lr': 'multilayer_perceptron', 'evaluation_test_ncf_low_weight_decay_increased_lr': 'neural_collaborative_filter', 'evaluation_test_cf_embed_all_except_tags_genres': 'collborative_filter_with_game_embeddings', 'evaluation_test_gcf_embed_all_except_tags_genres': 'non_linear_collaborative_filter_with_game_embeddings', 'evaluation_test_mlp_embed_all_except_tags_genres': 'multilayer_perceptron_with_game_embeddings', 'evaluation_test_ncf_embed_all_except_tags_genres': 'neural_collaborative_filter_with_game_embeddings'}
df = results.loc[results['model_save_path'].isin(evaluation_names), ['model_save_path', 'user_rank_auc_roc']].copy()
df['model_save_path'] = df['model_save_path'].map(evaluation_names)
display(df.sort_values(by='user_rank_auc_roc', ascending=False))

,model_save_path,user_rank_auc_roc
11,multilayer_perceptron_with_game_embeddings,0.916667
10,non_linear_collaborative_filter_with_game_embe...,0.562500
9,collborative_filter_with_game_embeddings,0.433333
6,non_linear_collaborative_filter,0.416667
8,neural_collaborative_filter,0.416667
5,collborative_filter,0.363636
12,neural_collaborative_filter_with_game_embeddings,0.293255
7,multilayer_perceptron,0.221719
